In [1]:
import cv2 
import numpy as np 
import os 
import matplotlib.pyplot as plt 

In [2]:
Xtrain = [] #array to store training and testing images
Ytrain = [] 

In [3]:
#Data preprocessing 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
classes = 0 
add = r"\train\g" #address of training data
original_data = 0
for data in os.listdir(add[:-2]):
    p = add[:-1] + data + "\g"
    original_data += len(os.listdir(p[:-1]))
    for d in os.listdir(p[:-1]):  
        img = cv2.imread(p[:-1] + d) 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        #To reduce the size of the input image for the model and removing the unwanted white space in the image. 
        black = np.column_stack(np.where(img == 0))
        x_values=[i[0] for i in black]
        y_values=[i[1] for i in black]
        xMin , xMax , yMin , yMax = min(x_values) , max(x_values) , min(y_values) , max(y_values)
        img = img[xMin - 10 : xMax + 10 , yMin - 10 : yMax + 10 ] #Creating an ROI around the text and leaving 
        #and leaving a margin of 10px
        img = cv2.resize(img, (28,28)) 
        img = img.reshape(1,img.shape[0],img.shape[1],1)
        datagen = ImageDataGenerator(rotation_range=90) #Image Augmentation to icrease the number of training data 
        it = datagen.flow(img, batch_size=1)
        for i in range(4):
            batch = it.next()
            image = batch[0].astype('uint8')
            Xtrain.append(image)
            Ytrain.append(classes)
    classes += 1 
        

In [7]:
print("Total number of Classes : " + str(classes))
print("Size of Training data before data augmentation: " + str(original_data))
print("Size of Training data after data augmentation: " + str(len(Xtrain)))

Total number of Classes : 62
Size of Training data before data augmentation: 2480
Size of Training data after data augmentation: 9920


In [9]:
#Data Prepration
from sklearn.utils import shuffle

Xtrain , Ytrain = shuffle(Xtrain , Ytrain) #Shuffling the data to create independent change in the data


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(Xtrain, Ytrain, test_size=0.33, random_state=42) #Spliting the data 
#into training and valditation data 

img_row = x_train[0].shape[0]
img_col = x_train[0].shape[1]

x_train  = np.array(x_train)
x_test  = np.array(x_test)

x_train=x_train.reshape(x_train.shape[0], img_row, img_col, 1)
x_test=x_test.reshape(x_test.shape[0], img_row, img_col, 1)

input_shape = (img_row, img_col, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255 #data Normalization
x_test /= 255

from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_train.shape[1]


In [13]:
#architecture:

import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , MaxPooling2D , Flatten,Dropout , BatchNormalization , Activation
from keras.optimizers import SGD

model = Sequential()

model.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,kernel_size=(3,3),activation = 'relu'))
model.add(BatchNormalization()) #To re-center and re-scale the layer
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3)) #To reduce over-fitting 

model.add(Conv2D(64,kernel_size=(3,3),activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(classes,activation = 'softmax'))

model.compile(loss='categorical_crossentropy',optimizer=SGD(0.01),metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 12, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 12, 12, 64)       

In [16]:
from keras.callbacks import ModelCheckpoint
filepath = r"\model1_1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [17]:
history = model.fit(x_train,y_train,batch_size=32,epochs=500, callbacks=callbacks_list ,verbose=1 , validation_data=(x_test,y_test))

Epoch 1/500
208/208 [==============================] - 8s 33ms/step - loss: 4.6144 - accuracy: 0.0294 - val_loss: 4.1270 - val_accuracy: 0.0183

Epoch 00001: val_accuracy improved from -inf to 0.01833, saving model to D:\Ananya\IIIT--D\HandWritingRecognition\model1_1.h5
Epoch 2/500
208/208 [==============================] - 7s 32ms/step - loss: 3.9481 - accuracy: 0.0562 - val_loss: 3.9810 - val_accuracy: 0.0574

Epoch 00002: val_accuracy improved from 0.01833 to 0.05742, saving model to D:\Ananya\IIIT--D\HandWritingRecognition\model1_1.h5
Epoch 3/500
208/208 [==============================] - 7s 33ms/step - loss: 3.7175 - accuracy: 0.0882 - val_loss: 3.5044 - val_accuracy: 0.1729

Epoch 00003: val_accuracy improved from 0.05742 to 0.17288, saving model to D:\Ananya\IIIT--D\HandWritingRecognition\model1_1.h5
Epoch 4/500
208/208 [==============================] - 7s 36ms/step - loss: 3.4783 - accuracy: 0.1237 - val_loss: 3.1521 - val_accuracy: 0.2269

Epoch 00004: val_accuracy improved f

In [19]:
model.save("model1_2.h5")